## **Imports**

In [1]:
%matplotlib inline
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader,Dataset
import matplotlib.pyplot as plt
import torchvision.utils
import numpy as np
import random
from PIL import Image
import cv2
import torch   
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from tqdm import tqdm
from torch.optim import lr_scheduler
import time
import copy

In [2]:
if torch.cuda.is_available():
  device = torch.device("cuda:0")
  print("Running on the GPU")
else: 
  device = torch.device("cpu")
  print("Running on the CPU")

Running on the GPU


In [3]:
# declate image size
imsize = 105

In [5]:
# Moun to Google Drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [6]:
! ls '/content/drive/My Drive/Mote_Manatee'

alldata_aug2.npy  jul27.pth	 out3.npy	paired_aug2.npy
jul23.pth	  MMLDUs_BatchA  out_jul23.npy


In [7]:
# Read the npy files
manatee_data = np.load('/content/drive/My Drive/Mote_Manatee/alldata_aug2.npy', allow_pickle=True)
manatee_paired = np.load('/content/drive/My Drive/Mote_Manatee/paired_aug2.npy', allow_pickle=True)

Manatee_paired is all the found pairs in the dataset. Manatee_data is all the images in the dataset

In [8]:
print('Manatee_data shape is: ', manatee_data.shape)
print('Manatee_paired data shape is: ',manatee_paired.shape)

Manatee_data shape is:  (1392, 105, 105)
Manatee_paired data shape is:  (726, 3)


In [9]:
# Change label from index-base to 1-0

for labeled_pair in manatee_paired:
  label = labeled_pair[2].argmax()
  labeled_pair[2] = label

## Make Data


Since we need genuine pairs for testing accuracy. I will reserve 10% of genuine pairs from manatee_paired.

In [10]:
# Reserve 10% of paired data for testing
num_points = round(manatee_paired.shape[0] * 0.1) #73
# set seed for reproducibility
np.random.seed(0)
# get 73 'random' numbers to use as indeces for manatee_paired data
indices = np.random.choice(manatee_paired.shape[0], num_points, replace=False)
test_paired = manatee_paired[indices]
test_paired.shape


(73, 3)

In [11]:
# get the remaining indices
indices_to_remove = indices
train_paired = np.delete(manatee_paired, indices_to_remove, axis=0)
train_paired.shape

(653, 3)

Now that the genuine pairs and all the images are available. I created this class to make the train-val dataset. The dataset will be a numpy array with each element being [img1, img2, label]. This dataset will include genuine and impostor pairs.

Note: make_dataset() will always return all the imporstor pairs followed by all the genuine pairs. It is important to shuffle the data once it has been created.

In [12]:
class MakeTrainValData():
  """
  This class is meant to make the train-val dataset by pairing impostor pairs and incorporating genuine pairs.
  """
  def __init__(self, train_paired, all_data):
    self.train_paired = train_paired
    self.all_data = all_data

  def make_dataset(self, pairs_per_image):
    data = []

    # make different pairs
    for img in self.all_data:
      img1 = img
      for i in range(pairs_per_image):
        random = np.random.randint(0, self.all_data.shape[0])
        img2 = self.all_data[random]
        label = 0

        data.append([img1, img2, label])

    # add labeled pairs
    for labeled_pair in self.train_paired:
      data.append(labeled_pair)

    return np.asarray(data)

In [13]:
data = MakeTrainValData(train_paired, manatee_data)
data = data.make_dataset(400)
print(f'Dataset has been created, with {data.shape[0]} labeled pairs')

Dataset has been created, with 557453 labeled pairs


In [14]:
# IMPORTANT! Shuffle dataset
np.random.seed(10)
shuffled_data = np.random.permutation(data)
shuffled_data.shape

(557453, 3)

## Separate train-val-test sets

In [15]:
val_num = round(shuffled_data.shape[0] * 0.1)
train_num = round(shuffled_data.shape[0] * 0.9)
print(f'Training data points: {train_num}')
print(f'Validation data points: {val_num}')

Training data points: 501708
Validation data points: 55745


In [44]:
###### GET TRAINING/VAL DATA POINTS

# set seed for reproducibility
np.random.seed(0)
# get 501708 'random' numbers to use as indeces for shuffled_data
indices = np.random.choice(shuffled_data.shape[0], train_num, replace=False)
train_data = shuffled_data[indices]
print(f'train_data shape is: {train_data.shape}')

# drop train indices from shuffled_data, which are the val_data points
indices_to_remove = indices
val_data = np.delete(shuffled_data, indices_to_remove, axis=0)
print(f'val_data shape is: {val_data.shape}')

train_data shape is: (501708, 3)
val_data shape is: (55745, 3)


## Compute mean and std for Training Data

In [52]:
class TrainDataNormalize(Dataset):
  def __init__(self, train_data, imsize=105, compute_mean_std=None):
    self.train_data = train_data
    self.imsize = imsize
    self.compute_mean_std = compute_mean_std

  def __len__(self):
    return self.train_data.shape[0]

  def __getitem__(self, idx):
    x1 = torch.Tensor(self.train_data[idx][0]).view(-1, self.imsize, self.imsize)
    x2 = torch.Tensor(self.train_data[idx][1]).view(-1, self.imsize, self.imsize)
    y = torch.Tensor([self.train_data[idx][2]])

    if self.compute_mean_std:
      return x1/255.0,x2/255.0
    else:
      return x1/255.0, x2/255.0, y

In [37]:
train_data_compute = TrainDataNormalize(train_data, compute_mean_std=True)

We need to manually implement the formulas for the mean and stantard deviation and iterate over small batches of the dataset.

source: https://deeplizard.com/learn/video/lu7TCu7HeYc

In [19]:
# First, we create a data loader with a small batch size
loader = DataLoader(train_data_compute, batch_size=1000, num_workers=1)

# Then, we calculate our n value or total number of pixels
num_of_pixels = train_num * 105 * 105   #Note that 105*105 is the height and width of the images in the dataset

# Now we sum the pixel values by iterating over each batch,
# and we calculate the mean by dividing this sum by the total num of pixels
total_sum = 0
for batch in loader: total_sum += batch[0].sum()
mean = total_sum/num_of_pixels

# Next we calculate the sum of the squared errors by iterating
# through each batch. This allows us to calculate st by dividing
# the sum of the squared errors by the total number of pixels and 
# square rooting the result

sum_of_squared_error = 0
for batch in loader:
  sum_of_squared_error += ((batch[0] - mean).pow(2)).sum()
std = torch.sqrt(sum_of_squared_error / num_of_pixels)


print(f'mean is {mean}, std is {std}')

mean is 0.9760749936103821, std is 0.12026530504226685


# **Dataset class loader**

In [64]:
class TrainValDataset(Dataset):
  """ Manatee Dataset Class """

  def __init__(self, manatee_data, imsize, transform=None):
    """
    Args: 
        manatee_data (np array): numpy structured as (x1,x2,y)
        transform (optional): optional transform to be applied on a sample
    """
    self.manatee_data = manatee_data
    self.transform = transform
    self.imsize = imsize
    #self.mean = mean 
    #self.std = std

  def __len__(self):
    return self.manatee_data.shape[0]

  def __getitem__(self, idx):
    x1 = torch.Tensor(self.manatee_data[idx][0]).view(-1, self.imsize, self.imsize)
    x2 = torch.Tensor(self.manatee_data[idx][1]).view(-1, self.imsize, self.imsize)
    y = torch.Tensor([self.manatee_data[idx][2]])

    if self.transform:
      x1_normal = x1/255.0
      x2_normal = x2/255.0
      x1 = self.transform(x1_normal)
      x2 = self.transform(x2_normal)

    return x1, x2, y

In [67]:
train_data_class = TrainValDataset(train_data, 105, transform=transforms.Normalize(mean=[mean], std=[std]))

In [68]:
# Train data class example:
train_data_class.__getitem__(0)

(tensor([[[0.1989, 0.1989, 0.1989,  ..., 0.1989, 0.1989, 0.1989],
          [0.1989, 0.1989, 0.1989,  ..., 0.1989, 0.1989, 0.1989],
          [0.1989, 0.1989, 0.1989,  ..., 0.1989, 0.1989, 0.1989],
          ...,
          [0.1989, 0.1989, 0.1989,  ..., 0.1989, 0.1989, 0.1989],
          [0.1989, 0.1989, 0.1989,  ..., 0.1989, 0.1989, 0.1989],
          [0.1989, 0.1989, 0.1989,  ..., 0.1989, 0.1989, 0.1989]]]),
 tensor([[[0.1989, 0.1989, 0.1989,  ..., 0.1989, 0.1989, 0.1989],
          [0.1989, 0.1989, 0.1989,  ..., 0.1989, 0.1989, 0.1989],
          [0.1989, 0.1989, 0.1989,  ..., 0.1989, 0.1989, 0.1989],
          ...,
          [0.1989, 0.1989, 0.1989,  ..., 0.1989, 0.1989, 0.1989],
          [0.1989, 0.1989, 0.1989,  ..., 0.1989, 0.1989, 0.1989],
          [0.1989, 0.1989, 0.1989,  ..., 0.1989, 0.1989, 0.1989]]]),
 tensor([0.]))

In [69]:
val_data_class = TrainValDataset(val_data, 105, transform=transforms.Normalize(mean=[mean], std=[std]))

In [70]:
# Val data class example:
val_data_class.__getitem__(0)

(tensor([[[0.1989, 0.1989, 0.1989,  ..., 0.1989, 0.1989, 0.1989],
          [0.1989, 0.1989, 0.1989,  ..., 0.1989, 0.1989, 0.1989],
          [0.1989, 0.1989, 0.1989,  ..., 0.1989, 0.1989, 0.1989],
          ...,
          [0.1989, 0.1989, 0.1989,  ..., 0.1989, 0.1989, 0.1989],
          [0.1989, 0.1989, 0.1989,  ..., 0.1989, 0.1989, 0.1989],
          [0.1989, 0.1989, 0.1989,  ..., 0.1989, 0.1989, 0.1989]]]),
 tensor([[[0.1989, 0.1989, 0.1989,  ..., 0.1989, 0.1989, 0.1989],
          [0.1989, 0.1989, 0.1989,  ..., 0.1989, 0.1989, 0.1989],
          [0.1989, 0.1989, 0.1989,  ..., 0.1989, 0.1989, 0.1989],
          ...,
          [0.1989, 0.1989, 0.1989,  ..., 0.1989, 0.1989, 0.1989],
          [0.1989, 0.1989, 0.1989,  ..., 0.1989, 0.1989, 0.1989],
          [0.1989, 0.1989, 0.1989,  ..., 0.1989, 0.1989, 0.1989]]]),
 tensor([0.]))

# **Neural Net Definition**

In [90]:
class Siamese(nn.Module):

    def __init__(self):
        super(Siamese, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 64, 10),  # 64@96*96
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),  # 64@48*48
            nn.Conv2d(64, 128, 7),
            nn.ReLU(),    # 128@42*42
            nn.MaxPool2d(2),   # 128@21*21
            nn.Conv2d(128, 128, 4),
            nn.ReLU(), # 128@18*18
            nn.MaxPool2d(2), # 128@9*9
            nn.Conv2d(128, 256, 4),
            nn.ReLU(),   # 256@6*6
        )
        self.linear = nn.Sequential(nn.Linear(9216, 4096)) #512 out
        #self.dropout = nn.Dropout(0.5)
        self.out = nn.Sequential(nn.Linear(4096, 1), nn.Sigmoid())

    def forward2(self, x1, x2):
      out1 = self.forward_one(x1)
      out2 = self.forward_one(x2)
      #L2_distance = torch.nn.PairwiseDistance(p=2, keepdim = True)
      #distance = L2_distance(out1, out2)
      #out = self.out(distance)
      cos = nn.CosineSimilarity(eps=1e-6)
      # similarity metric CV SDM
      out = cos(out1,out2)
      return out

    def forward_one(self, x):
        x = self.conv(x)
        x = x.view(x.size()[0], -1)
        out = self.linear(x)
        #output = self.dropout(out)
        return out

    def forward(self, input1, input2):
        out1 = self.forward_one(input1)
        out2 = self.forward_one(input2)
        L1_distance = torch.abs(out1 - out2)
        output = self.out(L1_distance)
        return output

In [101]:
net = Siamese()
# push to device
net = net.to(device)
print(sum(p.numel() for p in net.parameters()))

38951745


## Alternative Loss Function (Contrastive Loss)



In [ ]:
class ContrastiveLoss(torch.nn.Module):
    """
    Contrastive loss function.
    Based on: http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    """

    def __init__(self, margin=2.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, output1, output2, label):
        euclidean_distance = F.pairwise_distance(output1, output2, p=1, keepdim = True)
        loss_contrastive = torch.mean((1-label) * torch.pow(euclidean_distance, 2) +
                                      (label) * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2))

        return loss_contrastive

class ContrastiveLoss2(nn.Module):
    """
    Contrastive loss
    Takes embeddings of two samples and a target label == 1 if samples are from the same class and label == 0 otherwise
    """

    def __init__(self, margin):
        super(ContrastiveLoss2, self).__init__()
        self.margin = 0.2
        self.eps = 1e-9

    def forward(self, output1, output2, target, size_average=True):
        distances = (output2 - output1).pow(2).sum(1)  # squared distances
        losses = 0.5 * (target.float() * distances +
                        (1 + -1 * target).float() * F.relu(self.margin - (distances + self.eps).sqrt()).pow(2))
        return losses.mean() if size_average else losses.sum()

Margin in contrastive loss is more like regulirization constraint it defines hypersphere radius where all similar samples should locate .So from this point of view learning the margin will lead to overtraining “ bias-variance problem”. 

You can just normalize features using L2 before using Contrastive Loss. Then the margin can be constant while training because the distance between features will be normalized. You can of course change margin in dynamic way, but I do not think it will gain any boost. Idea from A Unified Embedding for Face Recognition and Clustering (although they use Triplet Loss, the same idea can be used in Siamese Network)

## Forward pass test example

In [93]:
train_loader_tst = DataLoader(train_data_class, batch_size=1, shuffle=True, num_workers=1)
net_tst = Siamese()

for x1, x2, label in train_loader_tst:
  out = net_tst(x1,x2)
  print(out)
  loss = torch.nn.functional.binary_cross_entropy(out, label)
  print(loss)
  break


tensor([[0.4955]], grad_fn=<SigmoidBackward>)
tensor(0.6843, grad_fn=<BinaryCrossEntropyBackward>)


# **Train**

In [94]:
#Create DataLoader iterators
train_loader = DataLoader(train_data_class, batch_size=64, shuffle=True, num_workers=2) 
val_loader = DataLoader(val_data_class, batch_size=64, shuffle=True, num_workers=2)

In [96]:
dataloaders = {'train': train_loader, 'val':val_loader}
dataset_sizes = {'train': train_data.shape[0], 'val': val_data.shape[0]}

In [98]:
def train_model(model, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = 0.0
    #best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            #running_corrects = 0

            # Iterate over data.
            for x1,x2,y in dataloaders[phase]:
                x1 = x1.to(device)
                x2 = x2.to(device)
                y = y.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # net.forward()
                    preds = model(x1,x2)
                    # compute loss
                    #loss = criterion(out1,out2, y)
                    loss = torch.nn.functional.binary_cross_entropy(preds, y)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * x1.size(0)
                #running_corrects += torch.sum(preds == y)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            #epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f}'.format(
                phase, epoch_loss)) #epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_loss < best_loss:
                best_loss = epoch_loss
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val loss: {:4f}'.format(best_loss))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [99]:
optimizer = optim.Adam(net.parameters(), lr = 0.05, weight_decay=1e-2) #lr = 0.00006)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)


In [103]:
# train and evaluate
model = train_model(net, optimizer, exp_lr_scheduler)

Epoch 0/24
----------


KeyboardInterrupt: ignored

## **Test Trained Network**

- Look for architecture (first)
- Bayesian optimization 
- data augmentation
- lr finder
- weight decay and L2 (optimizer)
- check Adam vs SGD with momentum with warm restarts (after lr finder)



In [ ]:
torch.save(model.state_dict, '/content/drive/My Drive/Mote_Manatee/jul27.pth')

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type Siamese. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [ ]:
! ls '/content/drive/My Drive/Mote_Manatee'

jul23.pth  jul27.pth  MMLDUs_BatchA  out3.npy  out_jul23.npy


In [ ]:
def support():
  """ 
  Test Average N-way oneshot learning accuracy
  """
  support_set = []


In [ ]:
val_tst = DataLoader(val_data, batch_size=32, shuffle=False, num_workers=2)

for x1, x2, label in val_tst:
  x1 = x1.to(device)
  x2 = x2.to(device)
  label = label.to(device)
  print(label)
  out = model.forward2(x1,x2)
  print(out)
  break

tensor([[1],
        [0],
        [0],
        [1],
        [0],
        [0],
        [1],
        [0],
        [0],
        [1],
        [0],
        [0],
        [1],
        [0],
        [0],
        [1],
        [0],
        [0],
        [1],
        [0],
        [0],
        [1],
        [0],
        [0],
        [1],
        [0],
        [0],
        [1],
        [0],
        [0],
        [1],
        [0]], device='cuda:0')
tensor([0.9981, 0.9961, 0.9957, 0.9990, 0.9990, 0.9986, 0.9983, 0.9955, 0.9973,
        0.9998, 0.9979, 0.9974, 0.9996, 0.9987, 0.9984, 0.9990, 0.9982, 0.9989,
        0.9988, 0.9989, 0.9971, 0.9977, 0.9973, 0.9966, 0.9965, 0.9958, 0.9958,
        0.9994, 0.9973, 0.9979, 0.9994, 0.9979], device='cuda:0',
       grad_fn=<DivBackward0>)


In [ ]:
for x1,x2,y val_data[0]:
  print(y)
  break

SyntaxError: ignored

In [ ]:
validation_data.shape

(165, 3)

In [ ]:
def oneshot(model, img1,img2, imsize=105):
  # Gives the feature vector of both inputs
  output1, output2 = model(img1.view(-1,1,imsize,imsize),img2.view(-1,1,imsize,imsize))
  # Compute the distance
  euclidean_distance = F.pairwise_distance(output1, output2)
  print(euclidean_distance)
